# <font color='red'>DO NOT MODIFY THIS FILE! MAKE A COPY</font>

# Binning from stored calibration settings and storing into a NeXus file
In this template, we use conversion parameters generated in another template, and use them to bin a scan into a nexus file.

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import os

import sed

%matplotlib widget

# Load Data

In [ ]:
# Path to the day folder containing the scan
data_path = "/mnt/pcshare/2025/2025_01/2025_01_09"
# the run to load for calibration
run = 664

In [ ]:
# Define an override config
config = {}
# Uncomment this line for data pre ~2020
# config = {"dataframe": {"tof_binning": 4, "adc_binning": 4}}
# Uncomment this line for data pre ~2024
# config = {"dataframe": {"tof_binning": 2, "adc_binning": 4}}

In [ ]:
metadata = {}
# Here, manual metadata can be added. Alternatively, they can come from an Electronic Lab Notebook below.
# Uncomment for adding manual Metadata (delete """-lines)
"""
#General
metadata['experiment_summary'] = ''
metadata['entry_title'] = ''
metadata['experiment_title'] = ''

#User
# Fill general parameters of NXuser
metadata['user0'] = {}
metadata['user0']['name'] = ''
metadata['user0']['role'] = 'Principal Investigator'
metadata['user0']['affiliation'] = 'Fritz Haber Institute of the Max Planck Society'
metadata['user0']['address'] = 'Faradayweg 4-6, 14195 Berlin'
metadata['user0']['email'] = '@fhi-berlin.mpg.de'

#NXinstrument
metadata['instrument'] = {}
#analyzer
metadata['instrument']['analyzer']={}
metadata['instrument']['analyzer']['slow_axes'] = "" # the scanned axes
metadata['instrument']['analyzer']['spatial_resolution'] = 10.
metadata['instrument']['analyzer']['energy_resolution'] = 110.
metadata['instrument']['analyzer']['momentum_resolution'] = 0.08
metadata['instrument']['analyzer']['working_distance'] = 4.

#probe beam
metadata['instrument']['beam']={}
metadata['instrument']['beam']['probe']={}
metadata['instrument']['beam']['probe']['incident_energy'] = 21.7
metadata['instrument']['beam']['probe']['incident_energy_spread'] = 0.04
metadata['instrument']['beam']['probe']['pulse_duration'] = 80.
metadata['instrument']['beam']['probe']['frequency'] = 475.
metadata['instrument']['beam']['probe']['incident_polarization'] = [1, 1, 0, 0] # p pol Stokes vector
metadata['instrument']['beam']['probe']['extent'] = [80., 80.] 
#pump beam
#metadata['instrument']['beam']['pump']={}
#metadata['instrument']['beam']['pump']['incident_energy'] = 1.55
#metadata['instrument']['beam']['pump']['incident_energy_spread'] = 0.08
#metadata['instrument']['beam']['pump']['pulse_duration'] = 35.
#metadata['instrument']['beam']['pump']['frequency'] = 475.
#metadata['instrument']['beam']['pump']['incident_polarization'] = [1, -1, 0, 0] # s pol Stokes vector
#metadata['instrument']['beam']['pump']['incident_wavelength'] = 800. 
#metadata['instrument']['beam']['pump']['average_power'] = 300.
#metadata['instrument']['beam']['pump']['pulse_energy'] = metadata['instrument']['beam']['pump']['average_power']/metadata['instrument']['beam']['pump']['frequency']#µJ
#metadata['instrument']['beam']['pump']['extent'] = [230., 265.] 
#metadata['instrument']['beam']['pump']['fluence'] = 0.15

#sample
metadata['sample']={}
metadata['sample']['preparation_date'] = ''
metadata['sample']['preparation_description'] = 'Cleaved'
metadata['sample']['sample_history'] = ''
metadata['sample']['chemical_formula'] = ''
metadata['sample']['description'] = ''
metadata['sample']['name'] = ''
""" and None

In [ ]:
# create sed processor using the config file, and collect the meta data from the files:
sp = sed.SedProcessor(folder=data_path, system_config="../config.yaml", runs=run, metadata=metadata, collect_metadata=True, verbose=True)
sp.add_jitter()

## Pose adjustment
Execute these cells if you want to perform manual pose adjustment

In [ ]:
# Calculate machine-coordinate data for pose adjustment
sp.bin_and_load_momentum_calibration(df_partitions=20, plane=145, width=5)

In [ ]:
# Adjust pose alignment, using stored distortion correction
# Carefully check the reported date of creation, and make sure you are using the right correction
sp.pose_adjustment(use_correction=True)
# Uncomment to use stored transformation parameters
# sp.pose_adjustment(use_correction=True, apply=True)

In [ ]:
# Apply stored momentum correction
sp.apply_momentum_correction()

In [ ]:
# Apply stored config momentum calibration
# Carefully check the reported creation data and make sure you are using the right calibration parameters
sp.apply_momentum_calibration()

In [ ]:
# Apply stored config energy correction
# Carefully check the reported creation data and make sure you are using the right correction parameters
sp.apply_energy_correction()

In [ ]:
# Apply stored config energy calibration
# Carefully check the reported creation data and make sure you are using the right calibration parameters
sp.append_energy_axis()

In [ ]:
# Apply stored delay calibration, or parameters from the file
delay_range = sp.calibrate_delay_axis(preview=True)

# Compute final data volume

In [ ]:
axes = ['kx', 'ky', 'energy', 'delay']
bins = [200, 200, 400, 100]
ranges = [[-2, 2], [-2, 2], [-3, 2], delay_range]
res = sp.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="delay")

In [ ]:
# save to NXmpes NeXus (including standardized metadata)
eln_data = f"Scan{run}.yaml"
save_path = f"Scan{run}_binned.nxs"
if os.path.isfile(eln_data):
    sp.save(save_path, eln_data=eln_data)
else:
    sp.save(save_path)

In [ ]:
# Visualization (requires JupyterLab)
from jupyterlab_h5web import H5Web
H5Web(save_path)